In [ ]:
# Testing Cell
import aviary.api as av
from aviary.api import Settings
from aviary.utils.doctape import glue_variable, check_value

HEIGHT_ENERGY = av.EquationsOfMotion.HEIGHT_ENERGY
glue_variable('height_energy', HEIGHT_ENERGY.name, md_code=True)
glue_variable('2DOF', av.EquationsOfMotion.TWO_DEGREES_OF_FREEDOM.name, md_code=True)

PAYLOAD_RANGE = av.Settings.PAYLOAD_RANGE

str_alternate_snippet = f'```\n{PAYLOAD_RANGE}, {True}\n```'
glue_variable('payload_range_toggle', str_alternate_snippet, md_code=True)
glue_variable('payload_range', av.Settings.PAYLOAD_RANGE, md_code=True)


glue_variable('operating_mass', av.Mission.Summary.OPERATING_MASS, md_code=True)
glue_variable('total_payload_mass', av.Aircraft.CrewPayload.TOTAL_PAYLOAD_MASS, md_code=True)
glue_variable('design_gross_mass', av.Mission.Design.GROSS_MASS, md_code=True)
glue_variable('total_fuel_capacity', av.Aircraft.Fuel.TOTAL_CAPACITY, md_code=True)

file_path3 = av.get_path('models/aircraft/test_aircraft/aircraft_for_bench_FwFm.csv')

glue_variable('objectives_fuel', av.Mission.Objectives.FUEL, md_code=True)
glue_variable('objectives_range', av.Mission.Objectives.RANGE, md_code=True)
glue_variable('summary_gross_mass', av.Mission.Summary.GROSS_MASS, md_code=True)

# Payload Range Functionality. 

## Overview

Aviary has the capability to generate payload range diagrams for models utilizing the {glue:md}`height_energy` equations of motion. Support for the {glue:md}`2DOF` equations of motion is planned in a future release.

A payload-range diagram is an aircraft performance visualization that illustrates the trade-off between how much mass the aircraft can carry in terms of passengers and cargo (payload) vs how far it can fly (range).
The payload-range shows 3 different points:
- **Maximum Payload Point ("Design Range")**: The range the aircraft can fly with maximum payload mass and fuel mass up to the maximum gross mass. This range is typically the design point of the aircraft (the "sizing" mission).
- **Maximum Fuel Point ("Max Economic Range")**: The range the aircraft can fly at maximum fuel capacity and payload mass up to maximum gross mass. This is the furthest range the aircraft can fly at its maximum gross mass.
- **Maximum Range Point ("Ferry Range")**: The range the aircraft can fly at maximum fuel capacity and zero payload. This is the furthest the aircraft can fly. 

![Payload Range Envelope Example](images/payload-range-envelope.png)

After a successful sizing mission (**Maximum Payload Point**) the payload-range function calculates the required aircraft loading conditions and runs an off-design fallout mission for the "Max Economic" "Ferry" range points.

When a user requests for a payload-range diagram to be generated, the additional points are run using the design mission's phase info. It is assumed that this `phase_info` contains a cruise phase named `cruise`, and Aviary will automatically adjust the `time_duration_bound` of the cruise phase to account for the longer off-design missions.

```{note}
We highly recommend that users first read through [Off Design Missions](off_design_missions) to understand the underlying mechanics of how Aviary runs off-design missions.
```

Each point is configured using the following methods:
- **Maximum Payload Point**: The results of the sizing mission are used to plot this point.

- **Maximum Fuel Point**: The aircraft's mission fuel is assumed to match {glue:md}`total_fuel_capacity`. The available payload is computed as {glue:md}`design_gross_mass` minus {glue:md}`operating_mass` minus {glue:md}`total_fuel_capacity`. Therefore the aircraft's mission mass is equal to its design gross mass.

- **Maximum Range Point**: The aircraft is flown at a gross mass equal to {glue:md}`design_gross_mass` plus {glue:md}`total_fuel_capacity`.

```{note}
There is an edge case where an aircraft's fuel capacity could exceed its design gross mass if fully fueled to that capacity. In this case, at the maximum fuel point the aircraft is filled with fuel up to its design gross mass. Because there is no room for additional payload, this point is also equivalent to the maximum range point. Aviary will not run that last point as it is redundant, and instead re-use the results of the maximum fuel point for building the payload-range data table.
```

## Outputs
After running all of the points of the payload-range diagram, a table of gross mass vs. range for each point is saved in a filed called `payload_range_data.csv`. This file is placed in the reports folder for that run.

## Running a payload/range analysis 
There are currently two methods to run a payload-range analysis. 
The first method is to set {glue:md}`payload_range` to True in your csv input file or AviaryValues input (if using the Level 2 interface).

```{note}
Setting this flag will result in Aviary running a payload-range analysis every time that specific aircraft model is sized. 
```

### Method Call
The second way to generate the payload-range diagram is to manually call the `run_payload_range()` method after running a sizing mission. If the payload range points run successfully, then the method returns a tuple containing the AviaryProblems used to create the maximum economic range and ferry range points.

`(max_econ_range_prob, ferry_range_prob) = prob.run_payload_range()`

## Output and visualization. 
The data for the payload-range points can be found in `payload_range_data.csv`, located within the main problem's reports folder. 
The information is also visualized within the `Payload/Range Diagram` tab in the Aviary dashboard for the sizing mission.  

```{note}
Aviary will only save the results payload/range analysis (and therefore only plot the diagram in the dashboard) if the sizing and two off-design points converged to a valid solution.
```